<a href="https://colab.research.google.com/github/basmala875/Machine-Learning-Projects/blob/main/Markov_model_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
nayakganesh007_nike_sales_uncleaned_dataset_path = kagglehub.dataset_download('nayakganesh007/nike-sales-uncleaned-dataset')

print('Data source import complete.')


Using Colab cache for faster access to the 'nike-sales-uncleaned-dataset' dataset.
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nike-sales-uncleaned-dataset/Nike_Sales_Uncleaned.csv


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from scipy import stats

df= pd.read_csv("/kaggle/input/nike-sales-uncleaned-dataset/Nike_Sales_Uncleaned.csv")
df.head()

,Order_ID,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Discount_Applied,Revenue,Order_Date,Sales_Channel,Region,Profit
0,2000,Kids,Training,SuperRep Go,M,NaN,NaN,0.47,0.0,2024-03-09,Online,bengaluru,-770.45
1,2001,Women,Soccer,Tiempo Legend,M,3.0,4957.93,NaN,0.0,2024-07-09,Retail,Hyd,-112.53
2,2002,Women,Soccer,Premier III,M,4.0,NaN,NaN,0.0,NaN,Retail,Mumbai,3337.34
3,2003,Kids,Lifestyle,Blazer Mid,L,NaN,9673.57,NaN,0.0,04-10-2024,Online,Pune,3376.85
4,2004,Kids,Running,React Infinity,XL,NaN,NaN,NaN,0.0,2024/09/12,Retail,Delhi,187.89


In [ ]:
df.pop('Order_ID')
df.head()

,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Discount_Applied,Revenue,Order_Date,Sales_Channel,Region,Profit
0,Kids,Training,SuperRep Go,M,NaN,NaN,0.47,0.0,2024-03-09,Online,bengaluru,-770.45
1,Women,Soccer,Tiempo Legend,M,3.0,4957.93,NaN,0.0,2024-07-09,Retail,Hyd,-112.53
2,Women,Soccer,Premier III,M,4.0,NaN,NaN,0.0,NaN,Retail,Mumbai,3337.34
3,Kids,Lifestyle,Blazer Mid,L,NaN,9673.57,NaN,0.0,04-10-2024,Online,Pune,3376.85
4,Kids,Running,React Infinity,XL,NaN,NaN,NaN,0.0,2024/09/12,Retail,Delhi,187.89


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Gender_Category   2500 non-null   object 
 1   Product_Line      2500 non-null   object 
 2   Product_Name      2500 non-null   object 
 3   Size              1990 non-null   object 
 4   Units_Sold        1265 non-null   float64
 5   MRP               1246 non-null   float64
 6   Discount_Applied  832 non-null    float64
 7   Revenue           2500 non-null   float64
 8   Order_Date        1884 non-null   object 
 9   Sales_Channel     2500 non-null   object 
 10  Region            2500 non-null   object 
 11  Profit            2500 non-null   float64
dtypes: float64(5), object(7)
memory usage: 234.5+ KB


In [ ]:
df.isnull().sum()

,0
Gender_Category,0
Product_Line,0
Product_Name,0
Size,510
Units_Sold,1235
MRP,1254
Discount_Applied,1668
Revenue,0
Order_Date,616
Sales_Channel,0


In [ ]:
df.pop('Discount_Applied')
df.head()

,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Revenue,Order_Date,Sales_Channel,Region,Profit
0,Kids,Training,SuperRep Go,M,NaN,NaN,0.0,2024-03-09,Online,bengaluru,-770.45
1,Women,Soccer,Tiempo Legend,M,3.0,4957.93,0.0,2024-07-09,Retail,Hyd,-112.53
2,Women,Soccer,Premier III,M,4.0,NaN,0.0,NaN,Retail,Mumbai,3337.34
3,Kids,Lifestyle,Blazer Mid,L,NaN,9673.57,0.0,04-10-2024,Online,Pune,3376.85
4,Kids,Running,React Infinity,XL,NaN,NaN,0.0,2024/09/12,Retail,Delhi,187.89


**Logical Impulsing**

In [ ]:
# Fill Units_Sold using MRP and Revenue (approximation)
mask_units = df['Units_Sold'].isna() & df['MRP'].notna() & df['Revenue'].notna()
df.loc[mask_units, 'Units_Sold'] = df.loc[mask_units, 'Revenue'] / df.loc[mask_units, 'MRP']

# Fill MRP if missing using Revenue / Units_Sold
mask_mrp = df['MRP'].isna() & df['Revenue'].notna() & df['Units_Sold'].notna()
df.loc[mask_mrp, 'MRP'] = df.loc[mask_mrp, 'Revenue'] / df.loc[mask_mrp, 'Units_Sold']

print(df[['Units_Sold','MRP']].isna().sum())
df[['Units_Sold','MRP']]

Units_Sold    622
MRP           742
dtype: int64


,Units_Sold,MRP
0,NaN,NaN
1,3.0,4957.93
2,4.0,0.00
3,0.0,9673.57
4,NaN,NaN
...,...,...
2495,3.0,0.00
2496,-1.0,-0.00
2497,0.0,6647.60
2498,4.0,5358.70


In [ ]:
# Fill remaining missing Units_Sold and MRP using contextual medians
for c in ['Units_Sold', 'MRP']:
    # Fill by Product_Line first
    df[c] = df.groupby('Product_Line')[c].transform(lambda x: x.fillna(x.median()))
    # Then by Gender_Category (if still NaN)
    df[c] = df.groupby('Gender_Category')[c].transform(lambda x: x.fillna(x.median()))
    # Finally, fill any remaining NaNs with overall median
    df[c] = df[c].fillna(df[c].median())

print("Nulls after full contextual imputation:")
print(df[['Units_Sold', 'MRP']].isna().sum())
df[['Units_Sold', 'MRP']]


Nulls after full contextual imputation:
Units_Sold    0
MRP           0
dtype: int64


,Units_Sold,MRP
0,0.0,4348.86
1,3.0,4957.93
2,4.0,0.00
3,0.0,9673.57
4,0.0,4638.25
...,...,...
2495,3.0,0.00
2496,-1.0,-0.00
2497,0.0,6647.60
2498,4.0,5358.70


In [ ]:
df['Units_Sold'] = df['Units_Sold'].round(0).astype(int)
df['MRP'] = df['MRP'].round(0).astype(int)
df['Revenue'] = df['Revenue'].round(0).astype(int)
df['Profit'] = df['Profit'].round(0).astype(int)

# Method 1: Get second mode after excluding zeros
def get_second_mode(column):
    # Get value counts excluding zeros
    value_counts = column[column != 0].value_counts()

    # Check if there are at least 2 unique values
    if len(value_counts) >= 2:
        return value_counts.index[1]  # Second most common
    elif len(value_counts) == 1:
        return value_counts.index[0]  # Only one value available
    else:
        return None  # No non-zero values

# Get second modes
second_mode_units = get_second_mode(df['Units_Sold'])
second_mode_mrp = get_second_mode(df['MRP'])

# Fill 0 values with second mode
df.loc[df['Units_Sold'] == 0, 'Units_Sold'] = second_mode_units
df.loc[df['MRP'] == 0, 'MRP'] = second_mode_mrp

df['Revenue'] = df['Units_Sold'] * df['MRP']

df.head()

,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Revenue,Order_Date,Sales_Channel,Region,Profit
0,Kids,Training,SuperRep Go,M,2,4349,8698,2024-03-09,Online,bengaluru,-770
1,Women,Soccer,Tiempo Legend,M,3,4958,14874,2024-07-09,Retail,Hyd,-113
2,Women,Soccer,Premier III,M,4,4469,17876,NaN,Retail,Mumbai,3337
3,Kids,Lifestyle,Blazer Mid,L,2,9674,19348,04-10-2024,Online,Pune,3377
4,Kids,Running,React Infinity,XL,2,4638,9276,2024/09/12,Retail,Delhi,188


In [ ]:
#drop the profit column as we have made the best use of it and we don't need it in the model building
df.pop('Profit')

df.head()

,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Revenue,Order_Date,Sales_Channel,Region
0,Kids,Training,SuperRep Go,M,2,4349,8698,2024-03-09,Online,bengaluru
1,Women,Soccer,Tiempo Legend,M,3,4958,14874,2024-07-09,Retail,Hyd
2,Women,Soccer,Premier III,M,4,4469,17876,NaN,Retail,Mumbai
3,Kids,Lifestyle,Blazer Mid,L,2,9674,19348,04-10-2024,Online,Pune
4,Kids,Running,React Infinity,XL,2,4638,9276,2024/09/12,Retail,Delhi


**Normalizaion**

In [ ]:

print(df['Size'].unique())


['M' 'L' 'XL' '11' nan '9' '6' '12' '7' '10' '8']


In [ ]:

def size_to_label(x):
    if pd.isna(x):
        mode_size = df['Size'].mode()[0]  # mode()[0] gives the first mode
        return mode_size
    elif x in ['M', 'L', 'XL']:
        return x
    else:
        # Convert numeric strings to int

        num = int(x)

        # Map numbers to labels
        if num in [6,7]:
            return 'S'
        elif num in [8,9]:
            return 'M'
        elif num in [10,11]:
            return 'L'
        elif num == 12:
            return 'XL'

df['Size_Label'] = df['Size'].apply(size_to_label)
df['Size_Label']
print(df['Size_Label'].unique())



['M' 'L' 'XL' 'S']


In [ ]:

df.head()

,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Revenue,Order_Date,Sales_Channel,Region,Size_Label
0,Kids,Training,SuperRep Go,M,2,4349,8698,2024-03-09,Online,bengaluru,M
1,Women,Soccer,Tiempo Legend,M,3,4958,14874,2024-07-09,Retail,Hyd,M
2,Women,Soccer,Premier III,M,4,4469,17876,NaN,Retail,Mumbai,M
3,Kids,Lifestyle,Blazer Mid,L,2,9674,19348,04-10-2024,Online,Pune,L
4,Kids,Running,React Infinity,XL,2,4638,9276,2024/09/12,Retail,Delhi,XL


In [ ]:
df['Size'] = df['Size_Label']
df.pop('Size_Label')

df.isna().sum()

,0
Gender_Category,0
Product_Line,0
Product_Name,0
Size,0
Units_Sold,0
MRP,0
Revenue,0
Order_Date,616
Sales_Channel,0
Region,0


**Label Encoding**

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['Product_Line'] = encoder.fit_transform(df['Product_Line'])

#view the mapping
state_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print("State encoding mapping:")
print(state_mapping)


State encoding mapping:
{'Basketball': np.int64(0), 'Lifestyle': np.int64(1), 'Running': np.int64(2), 'Soccer': np.int64(3), 'Training': np.int64(4)}


In [ ]:
df.head()

,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Revenue,Order_Date,Sales_Channel,Region
0,Kids,4,SuperRep Go,M,2,4349,8698,2024-03-09,Online,bengaluru
1,Women,3,Tiempo Legend,M,3,4958,14874,2024-07-09,Retail,Hyd
2,Women,3,Premier III,M,4,4469,17876,NaN,Retail,Mumbai
3,Kids,1,Blazer Mid,L,2,9674,19348,04-10-2024,Online,Pune
4,Kids,2,React Infinity,XL,2,4638,9276,2024/09/12,Retail,Delhi


In [ ]:
encoder = LabelEncoder()
df['Gender_Category'] = encoder.fit_transform(df['Gender_Category'])

#view the mapping
state_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print("State encoding mapping:")
print(state_mapping)

State encoding mapping:
{'Kids': np.int64(0), 'Men': np.int64(1), 'Women': np.int64(2)}


In [ ]:
df.head()

,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Revenue,Order_Date,Sales_Channel,Region
0,0,4,SuperRep Go,M,2,4349,8698,2024-03-09,Online,bengaluru
1,2,3,Tiempo Legend,M,3,4958,14874,2024-07-09,Retail,Hyd
2,2,3,Premier III,M,4,4469,17876,NaN,Retail,Mumbai
3,0,1,Blazer Mid,L,2,9674,19348,04-10-2024,Online,Pune
4,0,2,React Infinity,XL,2,4638,9276,2024/09/12,Retail,Delhi


In [ ]:
encoder = LabelEncoder()
df['Sales_Channel'] = encoder.fit_transform(df['Sales_Channel'])

#view the mapping
state_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print("State encoding mapping:")
print(state_mapping)

State encoding mapping:
{'Online': np.int64(0), 'Retail': np.int64(1)}


In [ ]:
df.head()

,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Revenue,Order_Date,Sales_Channel,Region
0,0,4,SuperRep Go,M,2,4349,8698,2024-03-09,0,bengaluru
1,2,3,Tiempo Legend,M,3,4958,14874,2024-07-09,1,Hyd
2,2,3,Premier III,M,4,4469,17876,NaN,1,Mumbai
3,0,1,Blazer Mid,L,2,9674,19348,04-10-2024,0,Pune
4,0,2,React Infinity,XL,2,4638,9276,2024/09/12,1,Delhi


**Normalization for Order_Date column** :



In [ ]:
# Convert Order_Date to consistent datetime format
df["Order_Date"] = pd.to_datetime(
    df["Order_Date"],
    errors="coerce",           # invalid dates -> NaT
    infer_datetime_format=True # automatically detect formats
)

# Impute missing dates with median
median_date = df["Order_Date"].median()
df["Order_Date"].fillna(median_date, inplace=True)

# (Optional) verify
print(df["Order_Date"].head(10))

0   2024-03-09
1   2024-07-09
2   2024-08-01
3   2024-08-01
4   2024-08-01
5   2024-08-01
6   2024-08-01
7   2024-08-01
8   2024-08-01
9   2024-08-01
Name: Order_Date, dtype: datetime64[ns]


/tmp/ipython-input-204130399.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["Order_Date"] = pd.to_datetime(
/tmp/ipython-input-204130399.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Order_Date"].fillna(median_date, inplace=True)


**markov model**

In [ ]:
import numpy as np
import pandas as pd

# 4. Sort by Order_Date to establish the market sequence
df.sort_values(by='Order_Date', inplace=True)

# --- Markov Model Construction ---

# 1. Define states and sequence
states_sequence = df['Product_Line'].tolist()
states = df['Product_Line'].unique()
state_index = {state: i for i, state in enumerate(states)}
n_states = len(states)

# 2. Initialize the transition count matrix
transition_matrix = np.zeros((n_states, n_states))

# 3. Populate transition counts (N_ij = count of transitions from state i to state j)
for i in range(len(states_sequence) - 1):
    current_state = states_sequence[i]
    next_state = states_sequence[i+1]

    # Get indices for the matrix
    i_curr = state_index[current_state]
    j_next = state_index[next_state]

    # Increment count
    transition_matrix[i_curr, j_next] += 1

# 4. Normalize to get probabilities (P_ij = N_ij / sum(N_i))
# This is done by dividing each row by its sum.
row_sums = transition_matrix.sum(axis=1, keepdims=True)
# Handle rows where the sum is 0 (i.e., the last state in the sequence)
row_sums[row_sums == 0] = 1
transition_matrix_prob = transition_matrix / row_sums

# 5. Convert to a pandas DataFrame for a clean, labeled output
transition_df = pd.DataFrame(transition_matrix_prob,
                             index=states,
                             columns=states)

print("Markov Transition Probability Matrix (Current State to Next State):")
print(transition_df.round(4))

Markov Transition Probability Matrix (Current State to Next State):
        3       1       2       4       0
3  0.1504  0.1970  0.1907  0.2458  0.2161
1  0.1477  0.1936  0.2375  0.2096  0.2116
2  0.1857  0.2089  0.1814  0.2342  0.1899
4  0.2125  0.2051  0.1886  0.2051  0.1886
0  0.2411  0.1976  0.1502  0.2016  0.2095
